# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [12]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
# Age對Fare做群聚編碼
"""
Your Code Here
"""
df['Age']=df['Age'].fillna('None')
mean_df=df.groupby(['Age'])['Fare'].mean().reset_index()
mode_df=df.groupby(['Age'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Age'])['Fare'].median().reset_index()
max_df = df.groupby(['Age'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Age'])
temp = pd.merge(temp, median_df, how='left', on=['Age'])
temp = pd.merge(temp, max_df, how='left', on=['Age'])
temp.columns = ['Age', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']
temp

,Age,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,0.42,8.516700,8.5167,8.51670,8.5167
1,0.67,14.500000,14.5000,14.50000,14.5000
2,0.75,19.258300,19.2583,19.25830,19.2583
3,0.83,23.875000,18.7500,23.87500,29.0000
4,0.92,151.550000,151.5500,151.55000,151.5500
5,1,30.005957,11.1333,37.00420,46.9000
6,2,37.536250,26.0000,26.95000,151.5500
7,3,25.781950,15.9000,23.53750,41.5792
8,4,29.543330,11.1333,25.45000,81.8583
9,5,22.717700,12.4750,23.50415,31.3875


In [14]:
df = pd.merge(df, temp, how='left', on=['Age'])
df = df.drop(['Age'] , axis=1)
df.head()

,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,NaN,S,25.504781,7.2500,7.89580,151.5500
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C,62.751509,0.0000,31.38750,227.5250
2,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,NaN,S,19.086806,7.8542,9.58125,78.8500
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S,89.312500,8.0500,26.14375,512.3292
4,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,NaN,S,89.312500,8.0500,26.14375,512.3292


In [16]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

8 Numeric Features : ['Pclass', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']



,Pclass,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,1,0,7.2500,25.504781,7.2500,7.89580,151.5500
1,1,1,0,71.2833,62.751509,0.0000,31.38750,227.5250
2,3,0,0,7.9250,19.086806,7.8542,9.58125,78.8500
3,1,1,0,53.1000,89.312500,8.0500,26.14375,512.3292
4,3,0,0,8.0500,89.312500,8.0500,26.14375,512.3292


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [18]:

from sklearn.linear_model import LinearRegression

# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max'] , axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.10444159488459345

In [19]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.09708606333390986

In [ ]:
沒有